In [7]:
import time
import argparse
import pandas as pd
import numpy as np
from src.utils import Logger, Setting
import os
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import MSELoss
from torch.optim import SGD, Adam
import random
import easydict
import re
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import logging
import json

In [8]:
args = easydict.EasyDict({
    'data_path': './data/',  # Data path 설정
    'saved_model_path': './saved_models',  # Saved Model path 설정
    'model': 'CNN_FM',  # 학습 및 예측할 모델 선택 (None으로 초기화, 사용 전에 설정 필요)
    'data_shuffle': True,  # 데이터 셔플 여부 조정
    'test_size': 0.2,  # Train/Valid split 비율 조정
    'seed': 42,  # Seed 값 조정
    'use_best_model': True,  # 검증 성능이 가장 좋은 모델 사용 여부 설정

    # TRAINING OPTION
    'batch_size': 1024,  # Batch size 조정
    'epochs': 10,  # Epoch 수 조정
    'lr': 1e-3,  # Learning Rate 조정
    'loss_fn': 'RMSE',  # 손실 함수 변경 (MSE 또는 RMSE)
    'optimizer': 'ADAM',  # 최적화 함수 변경 (SGD 또는 ADAM)
    'weight_decay': 1e-6,  # Adam optimizer에서 정규화에 사용하는 값 조정

    # GPU
    'device': 'cuda',  # 학습에 사용할 Device 조정

    # FM, FFM, NCF, WDN, DCN Common OPTION
    'embed_dim': 16,  # FM, FFM, NCF, WDN, DCN에서 embedding시킬 차원 조정
    'dropout': 0.2,  # NCF, WDN, DCN에서 Dropout rate 조정
    'mlp_dims': (16, 16),  # NCF, WDN, DCN에서 MLP Network의 차원 조정

    # CNN_FM
    'cnn_embed_dim': 64,  # CNN_FM에서 user와 item에 대한 embedding시킬 차원 조정
    'cnn_latent_dim': 12,  # CNN_FM에서 user/item/image에 대한 latent 차원 조정

})

# LOGGER

In [9]:
class Setting:
    @staticmethod
    def seed_everything(seed):
        
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    def __init__(self):
        now = time.localtime()
        now_date = time.strftime('%Y%m%d', now)
        now_hour = time.strftime('%X', now)
        save_time = now_date + '_' + now_hour.replace(':', '')
        self.save_time = save_time

    def get_log_path(self, args):
        
        path = f'./log/{self.save_time}_{args.model}/'
        return path

    def get_submit_filename(self, args):
        
        filename = f'./submit/{self.save_time}_{args.model}.csv'
        return filename

    def make_dir(self,path):
        
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass
        return path


class Logger:
    def __init__(self, args, path):
        
        self.args = args
        self.path = path

        self.logger = logging.getLogger()
        self.logger.setLevel(logging.INFO)
        self.formatter = logging.Formatter('[%(asctime)s] - %(message)s')

        self.file_handler = logging.FileHandler(self.path+'train.log')
        self.file_handler.setFormatter(self.formatter)
        self.logger.addHandler(self.file_handler)

    def log(self, epoch, train_loss, valid_loss):
        
        message = f'epoch : {epoch}/{self.args.epochs} | train loss : {train_loss:.3f} | valid loss : {valid_loss:.3f}'
        self.logger.info(message)

    def close(self):
       
        self.logger.removeHandler(self.file_handler)
        self.file_handler.close()

    def save_args(self):
        
        argparse_dict = self.args.__dict__

        with open(f'{self.path}/model.json', 'w') as f:
            json.dump(argparse_dict,f,indent=4)

    def __del__(self):
        self.close()


In [10]:
setting = Setting()
log_path = setting.get_log_path(args)
setting.make_dir(log_path)

logger = Logger(args, log_path)
logger.save_args()

## Preprocessing

In [11]:
def image_vector(path):
    
    img = Image.open(path)
    scale = transforms.Resize((32, 32))
    tensor = transforms.ToTensor()
    img_fe = Variable(tensor(scale(img)))
    return img_fe

def process_img_data(df, books, user2idx, isbn2idx, train=False):

    books_ = books.copy()
    books_['isbn'] = books_['isbn'].map(isbn2idx)

    if train == True:
        df_ = df.copy()
    else:
        df_ = df.copy()
        df_['user_id'] = df_['user_id'].map(user2idx)
        df_['isbn'] = df_['isbn'].map(isbn2idx)

    df_ = pd.merge(df_, books_[['isbn', 'img_path']], on='isbn', how='left')
    df_['img_path'] = df_['img_path'].apply(lambda x: 'data/'+x)
    img_vector_df = df_[['img_path']].drop_duplicates().reset_index(drop=True).copy()
    data_box = []
    for idx, path in tqdm(enumerate(sorted(img_vector_df['img_path']))):
        data = image_vector(path)
        if data.size()[0] == 3:
            data_box.append(np.array(data))
        else:
            data_box.append(np.array(data.expand(3, data.size()[1], data.size()[2])))
    img_vector_df['img_vector'] = data_box
    df_ = pd.merge(df_, img_vector_df, on='img_path', how='left')
    return df_

## DATA LOAD


In [12]:
path = './data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
train = pd.read_csv(path + 'train_ratings.csv')
test = pd.read_csv(path + 'test_ratings.csv')
sub = pd.read_csv(path + 'sample_submission.csv')

ids = pd.concat([train['user_id'], sub['user_id']]).unique()
isbns = pd.concat([train['isbn'], sub['isbn']]).unique()

idx2user = {idx:id for idx, id in enumerate(ids)}
idx2isbn = {idx:isbn for idx, isbn in enumerate(isbns)}

user2idx = {id:idx for idx, id in idx2user.items()}
isbn2idx = {isbn:idx for idx, isbn in idx2isbn.items()}

train['user_id'] = train['user_id'].map(user2idx)
sub['user_id'] = sub['user_id'].map(user2idx)

train['isbn'] = train['isbn'].map(isbn2idx)
sub['isbn'] = sub['isbn'].map(isbn2idx)

img_train = process_img_data(train, books, user2idx, isbn2idx, train=True)
img_test = process_img_data(test, books, user2idx, isbn2idx, train=False)

data = {
        'train':train,
        'test':test,
        'users':users,
        'books':books,
        'sub':sub,
        'idx2user':idx2user,
        'idx2isbn':idx2isbn,
        'user2idx':user2idx,
        'isbn2idx':isbn2idx,
        'img_train':img_train,
        'img_test':img_test,
        }

129777it [00:59, 2198.14it/s]
52000it [00:23, 2193.10it/s]


# DATA SPLIT

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
                                                        data['img_train'][['user_id', 'isbn', 'img_vector']],
                                                        data['img_train']['rating'],
                                                        test_size=args.test_size,
                                                        random_state=args.seed,
                                                        shuffle=True
                                                       )
data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid

In [14]:
class Image_Dataset(Dataset):
    def __init__(self, user_isbn_vector, img_vector, label):
        self.user_isbn_vector = user_isbn_vector
        self.img_vector = img_vector
        self.label = label
    def __len__(self):
        return self.user_isbn_vector.shape[0]
    def __getitem__(self, i):
        return {
                'user_isbn_vector' : torch.tensor(self.user_isbn_vector[i], dtype=torch.long),
                'img_vector' : torch.tensor(self.img_vector[i], dtype=torch.float32),
                'label' : torch.tensor(self.label[i], dtype=torch.float32),
                }

train_dataset = Image_Dataset(
                                data['X_train'][['user_id', 'isbn']].values,
                                data['X_train']['img_vector'].values,
                                data['y_train'].values
                            )
valid_dataset = Image_Dataset(
                                data['X_valid'][['user_id', 'isbn']].values,
                                data['X_valid']['img_vector'].values,
                                data['y_valid'].values
                            )
test_dataset = Image_Dataset(
                                data['img_test'][['user_id', 'isbn']].values,
                                data['img_test']['img_vector'].values,
                                data['img_test']['rating'].values
                            )

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, num_workers=0, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=args.batch_size, num_workers=0, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, num_workers=0, shuffle=False)
data['train_dataloader'], data['valid_dataloader'], data['test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader

## CNN_FM Model

In [15]:
class FactorizationMachine(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.v = nn.Parameter(torch.rand(input_dim, latent_dim), requires_grad = True)
        self.linear = nn.Linear(input_dim, 1, bias=True)


    def forward(self, x):
        linear = self.linear(x)
        square_of_sum = torch.mm(x, self.v) ** 2
        sum_of_square = torch.mm(x ** 2, self.v ** 2)
        pair_interactions = torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        output = linear + (0.5 * pair_interactions)
        return output


# factorization을 통해 얻은 feature를 embedding 합니다.
class FeaturesEmbedding(nn.Module):
    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)


    def forward(self, x: torch.Tensor):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)


# 이미지 특징 추출을 위한 기초적인 CNN Layer를 정의합니다.
class CNN_Base(nn.Module):
    def __init__(self, ):
        super(CNN_Base, self).__init__()
        self.cnn_layer = nn.Sequential(
                                        nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=3, stride=2),
                                        nn.Conv2d(6, 12, kernel_size=3, stride=2, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=3, stride=2),
                                        )
    def forward(self, x):
        x = self.cnn_layer(x)
        x = x.view(-1, 12 * 1 * 1)
        return x


# 기존 유저/상품 벡터와 이미지 벡터를 결합하여 FM으로 학습하는 모델을 구현합니다.
class CNN_FM(torch.nn.Module):
    def __init__(self, args, data):
        super().__init__()
        self.field_dims = np.array([len(data['user2idx']), len(data['isbn2idx'])], dtype=np.uint32)
        self.embedding = FeaturesEmbedding(self.field_dims, args.cnn_embed_dim)
        self.cnn = CNN_Base()
        self.fm = FactorizationMachine(
                                        input_dim=(args.cnn_embed_dim * 2) + (12 * 1 * 1),
                                        latent_dim=args.cnn_latent_dim,
                                        )


    def forward(self, x):
        user_isbn_vector, img_vector = x[0], x[1]
        user_isbn_feature = self.embedding(user_isbn_vector)
        img_feature = self.cnn(img_vector)
        feature_vector = torch.cat([
                                    user_isbn_feature.view(-1, user_isbn_feature.size(1) * user_isbn_feature.size(2)),
                                    img_feature
                                    ], dim=1)
        output = self.fm(feature_vector)
        return output.squeeze(1)

In [16]:
import os
import tqdm
import wandb
import torch
import torch.nn as nn
from torch.nn import MSELoss
from torch.optim import SGD, Adam


class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.eps = 1e-6
    def forward(self, x, y):
        criterion = MSELoss()
        loss = torch.sqrt(criterion(x, y)+self.eps)
        return loss


def train(args, model, dataloader, logger, setting):
    minimum_loss = 999999999
    if args.loss_fn == 'MSE':
        loss_fn = MSELoss()
    elif args.loss_fn == 'RMSE':
        loss_fn = RMSELoss()
    else:
        pass
    if args.optimizer == 'SGD':
        optimizer = SGD(model.parameters(), lr=args.lr)
    elif args.optimizer == 'ADAM':
        optimizer = Adam(model.parameters(), lr=args.lr)
    else:
        pass

    for epoch in tqdm.tqdm(range(args.epochs)):
        model.train()
        total_loss = 0
        batch = 0

        for idx, data in enumerate(dataloader['train_dataloader']):
            
            x, y = [data['user_isbn_vector'].to(args.device), data['img_vector'].to(args.device)], data['label'].to(args.device)
            
            y_hat = model(x)
            loss = loss_fn(y.float(), y_hat)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batch +=1

            # Log train loss
            # wandb.log({"Train Loss": total_loss/batch})

        valid_loss = valid(args, model, dataloader, loss_fn)

        # Log valid loss
        # wandb.log({"Valid Loss": valid_loss})

        print(f'Epoch: {epoch+1}, Train_loss: {total_loss/batch:.3f}, valid_loss: {valid_loss:.3f}')
        logger.log(epoch=epoch+1, train_loss=total_loss/batch, valid_loss=valid_loss)
        if minimum_loss > valid_loss:
            minimum_loss = valid_loss
            os.makedirs(args.saved_model_path, exist_ok=True)
            torch.save(model.state_dict(), f'{args.saved_model_path}/{setting.save_time}_{args.model}_model.pt')
    logger.close()
    return model


def valid(args, model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    batch = 0

    for idx, data in enumerate(dataloader['valid_dataloader']):
        
        x, y = [data['user_isbn_vector'].to(args.device), data['img_vector'].to(args.device)], data['label'].to(args.device)
        
        y_hat = model(x)
        loss = loss_fn(y.float(), y_hat)
        total_loss += loss.item()
        batch +=1
    valid_loss = total_loss/batch
    return valid_loss


def test(args, model, dataloader, setting):
    predicts = list()
    if args.use_best_model == True:
        model.load_state_dict(torch.load(f'./saved_models/{setting.save_time}_{args.model}_model.pt'))
    else:
        pass
    model.eval()

    for idx, data in enumerate(dataloader['test_dataloader']):
        
        x, _ = [data['user_isbn_vector'].to(args.device), data['img_vector'].to(args.device)], data['label'].to(args.device)
        
        y_hat = model(x)
        predicts.extend(y_hat.tolist())
    return predicts

# MODEL LOAD

In [17]:
model = CNN_FM(args, data).to(args.device)

# MODEL TRAIN

In [18]:
model = train(args, model, data, logger, setting)

 10%|█         | 1/10 [00:13<02:00, 13.42s/it]

Epoch: 1, Train_loss: 2.873, valid_loss: 2.476
Epoch: 2, Train_loss: 1.919, valid_loss: 2.279


 30%|███       | 3/10 [00:38<01:28, 12.58s/it]

Epoch: 3, Train_loss: 1.660, valid_loss: 2.326


 40%|████      | 4/10 [00:50<01:14, 12.45s/it]

...

Epoch: 8, Train_loss: 1.423, valid_loss: 2.386


 90%|█████████ | 9/10 [01:52<00:12, 12.31s/it]

Epoch: 9, Train_loss: 1.402, valid_loss: 2.387


100%|██████████| 10/10 [02:04<00:00, 12.43s/it]

Epoch: 10, Train_loss: 1.381, valid_loss: 2.396


# PREDICT

In [19]:
predicts = test(args, model, data, setting)

submission = pd.read_csv(args.data_path + 'sample_submission.csv')
submission['rating'] = predicts

In [20]:
submission

,user_id,isbn,rating
0,11676,0002005018,7.126083
1,116866,0002005018,8.419352
2,152827,0060973129,8.262862
3,157969,0374157065,8.165571
4,67958,0399135782,8.503297
...,...,...,...
76694,278543,1576734218,7.840802
76695,278563,3492223710,8.574254
76696,278633,1896095186,6.365067
76697,278668,8408044079,5.097304
